# Training of the model for Thumb Classification

The goal of this notebook is to train a classification model that can detect thumb up and thumb down in video stream

This notebook has been run on Google Colab to take advantage of the GPU.


In [1]:
import numpy as np 
import os 
import shutil 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input as preprocess_input_vgg
from keras.layers import Dense, Dropout, Flatten
from keras.models import Model
from keras import optimizers
from keras.models import load_model

import seaborn as sns

Define the three classification category

In [2]:
NB_CLASSES = 3
THUMB_UP = '01'
THUMB_DOWN = '02'
OTHER = '03'
PATH_TO_ZIP = '/content/DSTI_Python_Labs/assets/data_train'
ZIP_FILE = 'thumbv3.zip'
IMAGES_RAW = 'images_raw_thumb'
IMAGES_SPLITED = 'images_keras_thumb'
MODEL_FILE = 'model_thumb_v24112020.h5'
SAVE_FOLDER = 'save_model_thumb'

## Get the training data from the github repository

In [3]:
!git clone https://github.com/EricKiennemann/DSTI_Python_Labs.git

'git' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
cd $PATH_TO_ZIP

[WinError 3] The system cannot find the path specified: '/content/DSTI_Python_Labs/assets/data_train'
c:\Users\User\Documents\GitHub\Fruit-Detection-with-Yolov4\assets


In [5]:
!mkdir $IMAGES_RAW

**Unzip the file training data file locally locally**

In [6]:
import zipfile
with zipfile.ZipFile(ZIP_FILE, 'r') as zip_ref:
    zip_ref.extractall(IMAGES_RAW)

FileNotFoundError: [Errno 2] No such file or directory: 'thumbv3.zip'

In [ ]:
!ls $IMAGES_RAW

## Prepare the files for the processing

Split the files into three datasets (folders) :
* "train" for training
* "valid" for validation
* "test" for test

In [ ]:
def TrainValidTestFruit(category):
	# Path to the directory where the original dataset was uncompressed
	original_dataset_dir = IMAGES_RAW

	# Directory where the three datasets will be stored
	base_dir = IMAGES_SPLITED

	os.mkdir(base_dir)

	# Directory for the training splits
	train_dir = os.path.join(base_dir, 'train')
	os.mkdir(train_dir)

	# Directory for the validation splits
	valid_dir = os.path.join(base_dir, 'valid')
	os.mkdir(valid_dir)

	# Directory for the test splits
	test_dir = os.path.join(base_dir, 'test')
	os.mkdir(test_dir)

	for cat in category:
		# Directories for training categories
		train_category_dir = os.path.join(train_dir, cat)
		os.mkdir(train_category_dir)

		# Directories for validation categories
		valid_category_dir = os.path.join(valid_dir, cat)
		os.mkdir(valid_category_dir)

		# Directories for test categories
		test_category_dir = os.path.join(test_dir, cat)
		os.mkdir(test_category_dir)

		data_folder = os.path.join(original_dataset_dir, cat)
		jpgfiles = os.listdir(data_folder)

		nb_images = len(jpgfiles)
		train_ratio = 0.75				# 75% of files for training
		validation_ratio = 0.15		# 15% of files for validation
		test_ratio = 0.10					# 10% of files for test

		dataX = np.arange(nb_images)
		# train is now 75% of the entire data set
		x_train, x_test = train_test_split(dataX, test_size=1 - train_ratio)

		# test is now 10% of the initial data set
		# validation is now 15% of the initial data set
		x_valid, x_test = train_test_split(x_test, test_size=test_ratio/(test_ratio + validation_ratio)) 

		# Copy the train files 
		fnames = [jpgfiles[i] for i in x_train]
		for fname in fnames:
			src = os.path.join(original_dataset_dir, cat, fname)
			dst = os.path.join(train_category_dir, fname)
			shutil.copyfile(src, dst)

		# Copy the validation files 
		fnames = [jpgfiles[i] for i in x_valid]
		for fname in fnames:
			src = os.path.join(original_dataset_dir, cat, fname)
			dst = os.path.join(valid_category_dir, fname)
			shutil.copyfile(src, dst)

		# Copy the test files 
		fnames = [jpgfiles[i] for i in x_test]
		for fname in fnames:
			src = os.path.join(original_dataset_dir, cat, fname)
			dst = os.path.join(test_category_dir, fname)
			shutil.copyfile(src, dst)


		# Sanity Check to ensure that Training, Validation and Test Folders have the expected number of images

		print('Number of Images in Training Directory is {} for category {}'.format(len(os.listdir(train_category_dir)),cat))
		print('Number of Images in Validation Directory is {} for category {}'.format(len(os.listdir(valid_category_dir)),cat))
		print('Number of Images in Test Directory is {} for category {}'.format(len(os.listdir(test_category_dir)),cat))

# Run the creation of the three datasets on our three labels
TrainValidTestFruit([THUMB_UP,THUMB_DOWN,OTHER])

The dataset is quit well balanced between 'thumb up' 517 images and 'thumb down' 593 images for training

## Building the Neural Network

We'll be using VGG16 model and the corresponding preprocessing function for the input images.

In [ ]:
# include_top=false => we only take the convolutional part not the classification part.
# The image standard size is (224,224)
base_model = VGG16(include_top=False, weights='imagenet', input_shape = (224,224,3))
base_model.summary()

Note that we have downloaded only a convolution part of the neural network. Let's add some dense layers on top of it.  
I choose a sigmoid activation in order to be able to dect more easelly when there is "nothing" in the screen. If the probability for both 'thumb up' and 'thumb down' are low it is likely that there is no thumb on the screen

In [ ]:
flatten = Flatten()(base_model.output)
dropout_1 = Dropout(0.25)(flatten)
fc_1 = Dense(128)(dropout_1)
dropout_2 = Dropout(0.5)(fc_1)
predictions = Dense(NB_CLASSES, activation="sigmoid", name='predictions')(dropout_2)

In [ ]:
model = Model(base_model.input, predictions)

**The final model structure**

In [ ]:
model.summary()

**Chosing the optimizer parameters and compiling the model**  
Categorical crossentropy is choosen for this multi label classification problem

In [ ]:
loss = 'categorical_crossentropy'
learning_rate = 0.001
optimizer = optimizers.SGD ## optimizers.SGD 
metrics = ['accuracy']

In [ ]:
model.compile(loss=loss,
              optimizer=optimizer(learning_rate),
              metrics=metrics)


## Data preparation
We will do data augmentation in order to have more data for the training.
We apply :
* rotation
* width shift
* height shift


In [ ]:
train_dir = os.path.join(IMAGES_SPLITED, "train")
val_dir = os.path.join(IMAGES_SPLITED, "valid")
test_dir = os.path.join(IMAGES_SPLITED, "test")

In [ ]:
# we'll resize images in correspondance to network input size
image_size = (224,224)

In [ ]:
# apply some data augmentation
#
train_datagen = ImageDataGenerator(rotation_range=15,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   fill_mode='nearest',
                                   preprocessing_function=preprocess_input_vgg
                                  )

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg) # for validation we don't need to augment

train_batchsize = 40
val_batchsize = 40

# this function takes images from folders and feeds to Imagedatagenerator
train_generator = train_datagen.flow_from_directory( 
        train_dir,
        target_size=image_size,
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        val_dir,
        target_size=image_size,
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

**The data generation is only applied to the train dataset**

We do have 1370 images for training (without data augmentation) and 273 images for validation

## Model training

Starting with a number of epoch equal to 100

In [ ]:
epochs = 60

In [ ]:
nb_train_steps = train_generator.samples // train_generator.batch_size
nb_val_steps = validation_generator.samples // validation_generator.batch_size

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=nb_train_steps,
      epochs=epochs,
      validation_data=validation_generator,
      validation_steps=nb_val_steps,
      verbose=1, #0
)

**The accuracy for training and validation dataset are good**

In [ ]:
print('training acc.:',history.history['accuracy'][-1])
print('val acc.:', (history.history['val_accuracy'])[-1])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
def plot_history(history):
    plt.figure(figsize=(12,6))
    plt.subplot(1,2,1)
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy %')
    plt.plot(history.epoch, np.array(history.history['accuracy']),
    label='Train Accuracy')
    plt.plot(history.epoch, np.array(history.history['val_accuracy']),
    label = 'Val Accuracy')
    plt.legend()
    plt.title('Accuracy for train and validation')
    plt.ylim([0, 1.1])
    plt.subplot(1,2,2)
    plt.xlabel('Epoch')
    plt.ylabel('loss')
    plt.plot(history.epoch, np.array(history.history['loss']),
    label='Train Loss')
    plt.plot(history.epoch, np.array(history.history['val_loss']),
    label = 'Validation Loss')
    plt.legend()
    plt.title('Loss for train and validation')
    plt.show()

In [ ]:
plot_history(history)

## Saving model

The model is saved to be used in the back end part of the web application

In [ ]:
os.makedirs(SAVE_FOLDER, exist_ok=True)

In [ ]:
model_path = os.path.join(SAVE_FOLDER, MODEL_FILE)

In [ ]:
model.save(model_path)

## Final test on the test Dataset

In [ ]:
model = load_model(model_path)

Apply the same preprocessing on the images as for validation dataset

In [ ]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg) 

Realize the prediction

In [ ]:
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=image_size,
        shuffle = False,
        class_mode='categorical',
        batch_size=1)

filenames = test_generator.filenames
nb_samples = len(filenames)

predict = model.predict(test_generator,steps=nb_samples)

The prediction has been done for 185 images

**See the result of the prediction**

In [ ]:
def show_classification_confusion_matrix(y_valid,y_fit,list_classes):
  print(classification_report(y_valid, y_fit,
                            target_names = list_classes))
  mat = confusion_matrix(y_valid, y_fit)
  sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=list_classes,
            yticklabels=list_classes)
  plt.xlabel('true label')
  plt.ylabel('predicted label')


In [ ]:
# choose the higher probability as the best prediction
y_pred = np.argmax(predict, axis=1)
classes = ["{:02d}".format(i) for i in range(1, 4)]

show_classification_confusion_matrix(test_generator.classes,y_pred,classes)

**All the images has been correctly predicted**

**The model is kept and will be use for the web application**

## Store the model file on a google account

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cp $model_path ../../../gdrive/'My Drive'